In [7]:
import pickle

filename = 'Output/5by5_Data9000'
with open(filename, "rb") as file:
    stat = pickle.load(file)
    
# Calculate link travel time
link = stat['data']['network'].copy()
link['link_flow'] = stat['link_flow']
link['link_cost'] = round(link['free_flow_time']*\
                        (1+link['b']*((link['link_flow']/link['capacity'])**1)), 2)
link.head()

,~,init_node,term_node,capacity,length,free_flow_time,b,power,speed,toll,link_type,;,link_id,link_flow,link_cost
0,NaN,1,2,2000,40,0.6,0.15,4,60,0,0,;,0,1778,0.68
1,NaN,2,1,2000,40,0.6,0.15,4,60,0,0,;,1,0,0.60
2,NaN,1,6,2000,40,0.6,0.15,4,60,0,0,;,2,729,0.63
3,NaN,6,1,2000,40,0.6,0.15,4,60,0,0,;,3,780,0.64
4,NaN,2,3,2000,40,0.6,0.15,4,60,0,0,;,4,3560,0.76


In [8]:
import pandas as pd 

# Calculate path travel time for each od pair
def calculate_path_cost(row):
    sum_time = 0
    for l in row:
        sum_time += link.at[l, 'link_cost']
    return round(sum_time, 2)

path_link = stat['data']['paths_link']
od = [k for k in path_link.keys()]
path1 = [tuple(p[0]) for p in path_link.values()]
path2 = [tuple(p[1]) for p in path_link.values()]
path3 = [tuple(p[2]) for p in path_link.values()]

demand_dic = stat["data"]["demand"]
demand = [v for v in demand_dic.values()]

path_link_df = pd.DataFrame({"od": od, "demand":demand, "path1": path1, "path2": path2, "path3": path3})

path_link_df['path1_cost'] = path_link_df['path1'].apply(calculate_path_cost)
path_link_df['path2_cost'] = path_link_df['path2'].apply(calculate_path_cost)
path_link_df['path3_cost'] = path_link_df['path3'].apply(calculate_path_cost)

flows = stat['path_flow']
path_link_df['flow1'] = [f[0] for f in flows]
path_link_df['flow2'] = [f[1] for f in flows]
path_link_df['flow3'] = [f[2] for f in flows]

path_link_df.head()

,od,demand,path1,path2,path3,path1_cost,path2_cost,path3_cost,flow1,flow2,flow3
0,"(11, 19)",411,"(36, 40, 44, 50)","(38, 54, 58, 62)","(36, 42, 58, 62)",2.55,2.56,2.59,411,0,0
1,"(4, 12)",1469,"(9, 5, 6, 24)","(14, 27, 23, 24)","(9, 10, 23, 24)",2.79,2.78,2.79,376,815,278
2,"(14, 3)",626,"(33, 15, 9)","(45, 29, 11)","(33, 27, 11)",2.01,2.00,2.00,563,63,0
3,"(4, 18)",1024,"(9, 10, 28, 46)","(14, 27, 28, 46)","(14, 32, 45, 46)",2.72,2.71,2.70,0,0,1024
4,"(6, 9)",1361,"(18, 22, 26)","(3, 0, 4, 8, 14)","(18, 7, 4, 8, 14)",2.14,3.46,3.54,1361,0,0


In [18]:
from collections import defaultdict

path_flow = {}
for path_set, flow_set in zip(path_link.values(), flows):
    for path, flow in zip(path_set, flow_set):
        path_flow[tuple(path)] = flow

aggregated_sums = defaultdict(float)
for path, flow in path_flow.items():
    for link in path:
        aggregated_sums[link] += flow
link_flow = dict(aggregated_sums)
link_flow
# path_flow

{0: 20630.0,
 4: 35366.0,
 10: 13472.0,
 28: 20069.0,
 46: 23518.0,
 64: 14241.0,
 2: 13176.0,
 18: 22119.0,
 22: 34183.0,
 24: 22302.0,
 40: 31967.0,
 69: 16243.0,
 77: 9641.0,
 65: 17152.0,
 62: 32002.0,
 78: 12141.0,
 71: 5250.0,
 67: 0.0,
 33: 24427.0,
 15: 17683.0,
 9: 8883.0,
 45: 10744.0,
 29: 23888.0,
 11: 16334.0,
 27: 10218.0,
 5: 8197.0,
 6: 14646.0,
 42: 22692.0,
 14: 12135.0,
 23: 9367.0,
 54: 17560.0,
 39: 17530.0,
 36: 20438.0,
 56: 4806.0,
 72: 14871.0,
 61: 14171.0,
 41: 11594.0,
 58: 26693.0,
 26: 32416.0,
 32: 19545.0,
 50: 23726.0,
 68: 15148.0,
 44: 31904.0,
 51: 25478.0,
 63: 10542.0,
 47: 25685.0,
 74: 23735.0,
 76: 24375.0,
 66: 14855.0,
 70: 12070.0,
 8: 33762.0,
 12: 26216.0,
 16: 9753.0,
 30: 23814.0,
 57: 11480.0,
 43: 20395.0,
 7: 15178.0,
 20: 13473.0,
 19: 0.0,
 59: 10772.0,
 60: 17220.0,
 25: 21671.0,
 1: 0.0,
 37: 0.0,
 38: 11170.0,
 34: 15137.0,
 48: 19404.0,
 75: 8396.0,
 55: 0.0,
 21: 20010.0,
 3: 14251.0,
 73: 0.0,
 52: 19052.0,
 17: 5989.0,
 13: 0.

In [59]:
link_flow_df = pd.DataFrame(list(link_flow.items()), columns=['link_id', 'flow'])
khadija_flow = pd.DataFrame(enumerate(stat['link_flow']), columns=['link_id','khadija_flow'])
khadija_flow = pd.merge(khadija_flow, link_flow_df, on='link_id', how='outer')
khadija_flow['dif'] = khadija_flow['khadija_flow'] - khadija_flow['flow']
khadija_flow[khadija_flow['dif']!=0]

,link_id,khadija_flow,flow,dif
